In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
bcolz

<module 'bcolz' from '/usr/local/lib/python3.6/dist-packages/bcolz/__init__.py'>

In [3]:
PATH = "data/dogbreed/"

In [4]:
sz = 256
arch = dn201
bs = 32
# Jeremy tried 224, 58 and 299, 58

In [5]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv))) - 1
val_idxs = get_cv_idxs(n)

In [6]:
label_df = pd.read_csv(label_csv)

In [ ]:
label_df.pivot_table(index='breed', aggfunc=len).sort_values('id', ascending=False)

In [7]:
def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',
                                        val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)
    return data if sz>300 else data.resize(340, 'tmp')

## Precompute

In [8]:
data = get_data(sz, bs)

A Jupyter Widget

In [9]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

Downloading: "https://download.pytorch.org/models/densenet201-4c113574.pth" to /home/ubuntu/.torch/models/densenet201-4c113574.pth
100%|██████████| 81124970/81124970 [00:03<00:00, 23050776.07it/s]


100%|██████████| 324/324 [18:21<00:00,  3.40s/it]


In [10]:
learn.fit(1e-2, 3)

A Jupyter Widget

[ 0.       1.0539   0.57799  0.84012]                       
[ 1.       0.57837  0.44755  0.86307]                        
[ 2.       0.42784  0.41195  0.87088]                        



In [11]:
learn.fit(1e-2, 3)

A Jupyter Widget

[ 0.       0.33954  0.39627  0.8734 ]                        
[ 1.       0.26717  0.40159  0.86691]                        
[ 2.       0.21429  0.40803  0.866  ]                        



## Augment

In [12]:
from sklearn import metrics

In [13]:
data = get_data(sz, bs)

A Jupyter Widget

In [14]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [15]:
learn.fit(1e-2, 2)

A Jupyter Widget

[ 0.       1.24618  0.62005  0.83622]                       
[ 1.       0.76172  0.45081  0.87298]                        



In [16]:
learn.precompute=False
learn.bn_freeze=True

In [17]:
learn.fit(1e-2, 5, cycle_len=1)

ValueError: some parameters appear in more than one parameter group

In [ ]:
learn.save('224_ir2_pre')

In [ ]:
learn.load('224_ir2_pre')

## Increase size

In [ ]:
learn.set_data(get_data(350, bs))
learn.freeze()

In [ ]:
learn.fit(1e-2, 3, cycle_len=1)

In [21]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

A Jupyter Widget

[ 0.       0.26962  0.218    0.93103]                        
[ 1.       0.27629  0.21015  0.93534]                        
[ 2.       0.24661  0.20852  0.93391]                        
[ 3.       0.25752  0.20398  0.9387 ]                        
[ 4.       0.23323  0.20119  0.93534]                        
[ 5.       0.22318  0.19704  0.93918]                        
[ 6.       0.20048  0.196    0.9387 ]                        



In [22]:
log_preds, y = learn.TTA()
probs = np.exp(log_preds)
accuracy(log_preds, y), metrics.log_loss(y, probs)

(0.94422700587084152, 0.19929996627362331)

In [23]:
learn.save('299_pre')

In [24]:
learn.load('299_pre')

In [25]:
learn.fit(1e-2, 1, cycle_len=2)

A Jupyter Widget

[ 0.       0.21182  0.20144  0.9363 ]                        
[ 1.       0.20089  0.19496  0.93726]                        



In [26]:
learn.save('299_pre')

In [27]:
log_preds, y = learn.TTA(is_test=True)
probs_resnx101_64 = np.exp(log_preds)
#accuracy(log_preds, y), metrics.log_loss(y, probs)

In [28]:
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()

In [29]:
save_array('probs_resnx101_64.bc', probs_resnx101_64)

In [30]:
learn.save('299_pre')

## Use whole dataset now

In [31]:
learn.load('299_pre')

In [32]:
def get_data_whole(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',
                                        val_idxs=[0], suffix='.jpg', tfms=tfms, bs=bs)
    return data if sz>300 else data.resize(340, 'tmp')

In [33]:
def load_array(fname): return bcolz.open(fname)[:]

In [34]:
data = get_data_whole(sz, bs)

A Jupyter Widget

In [35]:
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.freeze()

100%|██████████| 1/1 [00:00<00:00,  2.82it/s]


In [36]:
learn.fit(1e-2, 5)

A Jupyter Widget

[ 0.       0.75786  0.01884  1.     ]                         
[ 1.       0.38308  0.00202  1.     ]                         
[ 2.       0.27529  0.00073  1.     ]                         
[ 3.       0.22052  0.00052  1.     ]                         
[ 4.       0.19112  0.00022  1.     ]                         



In [37]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [38]:
learn.fit(1e-2, 2)

A Jupyter Widget

[ 0.       0.94042  0.00524  1.     ]                         
[ 1.       0.48962  0.00265  1.     ]                         



In [39]:
learn.precompute = False
learn.bn_freeze = True

In [ ]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

A Jupyter Widget

[ 0.       0.44712  0.00127  1.     ]                        
[ 1.       0.39743  0.00121  1.     ]                        
[ 2.       0.34848  0.00075  1.     ]                        
[ 3.       0.35622  0.00137  1.     ]                        
[ 4.       0.31755  0.00057  1.     ]                        
[ 5.       0.28701  0.00074  1.     ]                        
 76%|███████▌  | 134/177 [03:41<01:10,  1.65s/it, loss=0.29] 

In [ ]:
learn.set_data(get_data_whole(299, bs))
learn.freeze()

In [ ]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

In [ ]:
log_preds, y = learn.TTA(is_test=True)
probs_resnx101_64_full = np.exp(log_preds)
#accuracy(log_preds, y), metrics.log_loss(y, probs)

In [ ]:
save_array('probs_resnx101_64_full.bc', probs_resnx101_64_full)

In [ ]:
learn.save('299_full')

## Try ensembling
### Use another arch first

## Save to file for submission

In [ ]:
df = pd.DataFrame(probs)
df.columns = data.classes
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [ ]:
df.head()

In [ ]:
SUBM = f'{PATH}results_2/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}submission7.csv', index=False)
#df.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

## Individual prediction

In [ ]:
fn = data.val_ds.fnames[0]

In [ ]:
fn

In [ ]:
Image.open(PATH+fn).resize((150, 150))

In [ ]:
trn_tfms, val_tfms = tfms_from_model(arch, sz)

In [ ]:
ds = FilesIndexArrayDataset([fn], np.array([0]), val_tfms, PATH)
dl = DataLoader(ds)
preds = learn.predict_dl(dl)
np.argmax(preds)

In [ ]:
im = trn_tfms(Image.open(PATH+fn))
preds = to_np(learn.model(V(T(im[None]).cude())))
np.argmax(preds)

In [ ]:
trn_tfms, val_tfms = tfms_from_model(arch, sz)

In [ ]:
im = trn_tfms(Image.open(PATH+fn))
preds = learn.predict_array(im[None])
np.argmax(preds)

## Confusion matrix

In [ ]:
preds = np.argmax(log_preds, axis=1)
probs = np.exp(log_preds[:,1])

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

In [ ]:
plot_confusion_matrix(cm, data.classes)